#TrustPilot



In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from google.colab import files

base_url = "https://au.trustpilot.com/review/opencolleges.edu.au?page="

#Web Scraper using BeautifulSoup
def extract_review_data(review):
   #Rating
    try:
        rating = int(review.find("div", {"data-service-review-rating": True})["data-service-review-rating"])
    except (AttributeError, TypeError):
        rating = None
    #Date
    try:
        date = review.find("time")["datetime"]
    except (AttributeError, TypeError):
        date = None
    #Title
    try:
        title = review.find("h2").get_text(strip=True)
    except (AttributeError, TypeError):
        title = None
    #Review
    try:
        review_text = review.find("p", {"data-service-review-text-typography": "true"}).get_text(strip=True)
    except (AttributeError, TypeError):
        review_text = None
    #Join info
    return {
        "rating": rating,
        "date": date,
        "title": title,
        "review": review_text
    }

all_reviews = []

#Crawler
for page in range(1,167):  # 166 pages as inspected
    url = base_url + str(page)
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews = soup.find_all("section", class_="styles_reviewContentwrapper__zH_9M")

        for review in reviews:
            review_data = extract_review_data(review)
            all_reviews.append(review_data)
    else:
        print(f"Failed to fetch page {page}") #Only for Trust pilot as pages are numerous 

    time.sleep(4)  # Add a delay of 4 second between requests to avoid overloading the server

# Create a dataframe and save the data
df = pd.DataFrame(all_reviews)
df.to_csv("trustpilot_reviews.csv", index=False)

# Download the file using Google Colab
files.download("trustpilot_reviews.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ProductReviews

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from google.colab import files

base_url = "https://www.productreview.com.au/listings/opencolleges?page="

#Web Scraper using BeautifulSoup
def extract_review_data(review):
    #Rating
    star = review.find('div', {'title': re.compile(r'\d out of 5 stars')})['title']
    #Date
    date = review.find('time')['datetime']
    #Title
    title = review.find('h3', {'data-layout': 'title'}).text.strip()
    #Review, two options in case of extended reviews
    review_text = ""
    review_body = review.find('div', {'itemprop': 'reviewBody'})
    if review_body:
        review_text = review_body.text.strip()
    else:
        review_text_span = review.find('span', {'itemprop': 'reviewBody'})
        if review_text_span:
            review_text = review_text_span.text.strip()

    return {
        'star': star,
        'date': date,
        'title': title,
        'review': review_text
    }

all_reviews = []

#Crawler - second method intended - not delimiting the pages, but allow to run wile getting a positive response
page_number = 1
while True:
    response = requests.get(base_url + str(page_number))
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = soup.find_all('div', class_='_UXni3 kTRKyO Owvr8w V_nCZG f628e7')

    if not reviews:
        break

    for review in reviews:
        all_reviews.append(extract_review_data(review))

    page_number += 1

reviews_df = pd.DataFrame(all_reviews)
reviews_df.to_csv('Productreviews1.csv', index=False)
files.download('Productreviews1.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>